![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 4.Training and Reusing Text Classification Models

**Relevant blogpost:** https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32

In [0]:
import json
import os
import string
import pandas as pd
import numpy as np

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *

import pyspark.sql.functions as F
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 100)

print('sparknlp.version : ',sparknlp.version())

spark

sparknlp.version : 4.3.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## Using Pretrained ClassifierDL and SentimentDL models

In [0]:
fake_classifier = ClassifierDLModel.pretrained('classifierdl_use_fakenews', 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

fake_news classifier is trained on `https://raw.githubusercontent.com/joolsa/fake_real_news_dataset/master/fake_or_real_news.csv.zip`

In [0]:
fake_classifier.getClasses()

Out[3]: ['FAKE', 'REAL']

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use",lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      use,
      fake_classifier
  ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

fake_clf_model = nlpPipeline.fit(empty_data)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/spam_ham_dataset.csv
  
dbutils.fs.cp("file:/databricks/driver/spam_ham_dataset.csv", "dbfs:/") 

Out[5]: True

In [0]:
fake_lp_pipeline = LightPipeline(fake_clf_model)

text = 'BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'

fake_lp_pipeline.annotate(text)

Out[6]: {'document': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'sentence_embeddings': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'class': ['FAKE']}

In [0]:
sample_data = spark.createDataFrame([[text]]).toDF("text")

sample_data.show(truncate=False)

+-------------------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------------------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|
+-------------------------------------------------------------------------------------------------+

In [0]:
pred = fake_clf_model.transform(sample_data)


In [0]:
pred.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence_embeddings| class|
+--------------------+--------------------+--------------------+--------------------+
BREAKING: Leaked ...|[{document, 0, 96...|[{sentence_embedd...|[{category, 0, 96...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
pred.select('text','class.result').show(truncate=False)

+-------------------------------------------------------------------------------------------------+------+
text |result|
+-------------------------------------------------------------------------------------------------+------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|[FAKE]|
+-------------------------------------------------------------------------------------------------+------+

you can find more samples here >> `https://github.com/KaiDMML/FakeNewsNet/tree/master/dataset`

## Generic classifier function

In [0]:
def get_clf_lp(model_name, sentiment_dl=False, pretrained=True):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  use = UniversalSentenceEncoder.pretrained(lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")


  if pretrained:

    if sentiment_dl:

      document_classifier = SentimentDLModel.pretrained(model_name, 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

  else:

    if sentiment_dl:

      document_classifier = SentimentDLModel.load(model_name) \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.load(model_name) \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

  print ('classes:',document_classifier.getClasses())

  nlpPipeline = Pipeline(stages=[
                  documentAssembler, 
                  use,
                  document_classifier
  ])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  clf_pipelineFit = nlpPipeline.fit(empty_data)

  clf_lp_pipeline = LightPipeline(clf_pipelineFit)

  return clf_lp_pipeline


In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_trec50')

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_trec50 download started this may take some time.
Approximate size to download 21.2 MB
[ | ][ / ][ — ][OK!]
classes: [' ENTY_color', ' ENTY_techmeth', ' DESC_manner', ' NUM_volsize', ' ENTY_letter', ' NUM_temp', ' ENTY_body', ' NUM_count', ' ENTY_instru', ' NUM_period', ' NUM_speed', ' DESC_reason', ' ENTY_symbol', ' ENTY_event', ' HUM_desc', ' NUM_perc', ' ENTY_dismed', ' NUM_ord', ' HUM_gr', ' LOC_mount', ' ABBR_abb', ' DESC_desc', ' NUM_dist', ' HUM_title', ' ENTY_lang', ' ENTY_sport', ' ENTY_plant', ' NUM_code', ' NUM_other', ' ENTY_word', ' ENTY_animal', ' ENTY_substance', ' ENTY_veh', ' ENTY_product', ' LOC_state', ' ENTY_religion', ' ENTY_currency', ' NUM_date', ' LOC_country', ' ENTY_cremat', ' NUM_money', ' LOC_other', ' DESC_def', ' LOC_city', ' HUM_ind', ' ENTY_other', ' ENTY_termeq', ' ENTY_food', ' ABBR_exp', ' NUM_weight']

trained on the TREC datasets:

Classify open-domain, fact-based questions into one of the following broad semantic categories: 

```Abbreviation, Description, Entities, Human Beings, Locations or Numeric Values.```

In [0]:
text = 'What was the number of member nations of the U.N. in 2000?'

clf_lp_pipeline.annotate(text)['class']

Out[13]: [' NUM_count']

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].result

Out[14]: ' NUM_count'

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].metadata

Out[15]: {' ENTY_dismed': '3.768739E-22', ' ENTY_product': '2.4015744E-24', ' ENTY_techmeth': '1.5787039E-22', ' NUM_speed': '7.948464E-23', ' NUM_volsize': '2.5315113E-25', ' LOC_state': '6.3784123E-25', ' NUM_code': '1.4549451E-25', ' NUM_count': '0.9992601', ' ENTY_food': '1.3031208E-24', ' ENTY_animal': '1.6743833E-24', ' NUM_period': '6.8075115E-21', ' ENTY_religion': '5.9194734E-23', ' LOC_country': '5.3062683E-21', ' LOC_mount': '3.2177816E-25', ' ENTY_termeq': '9.790085E-26', ' ENTY_color': '1.1446835E-22', ' ENTY_lang': '6.333391E-24', ' ENTY_sport': '8.0773835E-25', ' DESC_def': '2.4284432E-27', ' HUM_gr': '4.4863106E-21', ' ENTY_symbol': '4.1271923E-25', ' ENTY_currency': '8.156541E-29', ' ENTY_veh': '5.414701E-22', ' LOC_other': '5.5141072E-11', ' ENTY_word': '5.3265024E-23', ' NUM_temp': '2.0907158E-23', ' NUM_dist': '1.2542656E-24', ' DESC_desc': '1.0926973E-12', ' DESC_manner': '9.258374E-23', ' NUM_ord': '2.2395288E-25', ' NUM_other': '3.9771262E-27', ' DESC_reason': '1.1718967E-6', ' NUM_weight': '1.5373857E-24', ' ENTY_instru': '5.9354656E-21', ' ENTY_letter': '1.1453239E-25', ' ENTY_event': '3.706315E-25', ' ENTY_substance': '6.890844E-25', ' ABBR_exp': '5.6048268E-24', ' ENTY_body': '6.423101E-23', ' ENTY_other': '7.378E-4', ' NUM_money': '1.6745677E-25', ' LOC_city': '4.7003377E-22', ' NUM_date': '5.2122506E-16', ' NUM_perc': '6.3761288E-24', ' ABBR_abb': '7.101014E-26', ' ENTY_plant': '5.543376E-24', ' HUM_title': '1.0681953E-24', ' ENTY_cremat': '1.1165376E-24', ' HUM_ind': '8.063818E-7', ' HUM_desc': '4.3701275E-23', 'sentence': '0'}

In [0]:
text = 'What animal was the first mammal successfully cloned from adult cells?'

clf_lp_pipeline.annotate(text)['class']

Out[16]: [' HUM_ind']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_cyberbullying')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.3 MB
[ | ][ / ][ — ][OK!]
classes: ['sexism', 'neutral', 'racism']

In [0]:
text ='RT @EBeisner @ahall012 I agree with you!! I would rather brush my teeth with sandpaper then watch football with a girl!!'

clf_lp_pipeline.annotate(text)['class']

Out[18]: ['sexism']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_fakenews')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][OK!]
classes: ['FAKE', 'REAL']

In [0]:
text ='Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton campaign accusation that Donald Trump is a KGB spy is about as weak and baseless a claim as a Salem witch hunt or McCarthy era trial. It’s only because Hillary Clinton is losing that she is lobbing conspiracy theory. Citizen Quasar The way I see it, one of two things will happen: 1. Trump will win by a landslide but the election will be stolen via electronic voting, just like I have been predicting for over a decade, and the American People will accept the skewed election results just like they accept the TSA into their crotches. 2. Somebody will bust a cap in Hillary’s @$$ killing her and the election will be postponed. Follow AMTV!'

clf_lp_pipeline.annotate(text)['class']


Out[20]: ['FAKE']

In [0]:
text ='Sen. Marco Rubio (R-Fla.) is adding a veteran New Hampshire political operative to his team as he continues mulling a possible 2016 presidential bid, the latest sign that he is seriously preparing to launch a campaign later this year.Jim Merrill, who worked for former GOP presidential nominee Mitt Romney and ran his 2008 and 2012 New Hampshire primary campaigns, joined Rubio’s fledgling campaign on Monday, aides to the senator said.Merrill will be joining Rubio’s Reclaim America PAC to focus on Rubio’s New Hampshire and broader Northeast political operations."Marco has always been well received in New Hampshire, and should he run for president, he would be very competitive there," Terry Sullivan, who runs Reclaim America, said in a statement. "Jim certainly knows how to win in New Hampshire and in the Northeast, and will be a great addition to our team at Reclaim America.”News of Merrill’s hire was first reported by The New York Times.'

clf_lp_pipeline.annotate(text)['class']

Out[21]: ['REAL']

In [0]:
sentiment_lp_pipeline = get_clf_lp('sentimentdl_use_twitter', sentiment_dl=True)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ][ / ][ — ][OK!]
classes: ['positive', 'negative']

In [0]:
text ='I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!'

sentiment_lp_pipeline.annotate(text)['class']

Out[23]: ['positive']

In [0]:
sentiment_lp_pipeline = get_clf_lp('classifierdl_use_emotion', sentiment_dl=False)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[ | ][ / ][ — ][ \ ][OK!]
classes: ['joy', 'fear', 'surprise', 'sadness']

In [0]:
sentiment_lp_pipeline.annotate(text)['class']

Out[25]: ['surprise']

## ClassiferDL with Word Embeddings and Text Preprocessing

### Load Dataset

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv
  
dbutils.fs.cp("file:/databricks/driver/news_category_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/news_category_test.csv", "dbfs:/")

Out[26]: True

In [0]:
%sh cd /databricks/driver/ && ls -lt 

total 31624
-rw-r--r-- 1 root root 1504408 Mar 13 16:15 news_category_test.csv
-rw-r--r-- 1 root root 24032125 Mar 13 16:15 news_category_train.csv
drwxr-xr-x 2 root root 4096 Mar 13 16:15 ganglia
-rw-r--r-- 1 root root 5502589 Mar 13 16:13 spam_ham_dataset.csv
drwxr-xr-x 2 root root 4096 Mar 13 16:10 logs
drwxr-xr-x 3 root root 4096 Mar 13 16:10 eventlogs
-r-xr-xr-x 1 root root 1307454 Mar 13 14:15 preload_class.lst
drwxr-xr-x 2 root root 4096 Mar 13 14:15 azure
-r-xr-xr-x 1 root root 3037 Mar 13 14:15 hadoop_accessed_config.lst
drwxr-xr-x 2 root root 4096 Mar 13 14:15 conf

In [0]:
trainDataset = spark.read \
    .option("header", True) \
    .csv("/news_category_train.csv")\
    .sample(0.0001) # For demonstration purposes only sample a fraction of the dataset

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business| Oil prices rose on Friday as a sabotage attac...|
Sci/Tech|An Australian software developer has been left ...|
Sci/Tech|security companies TruSecure Corp. and Betruste...|
 Sports| Tennessee Titans quarterback Steve McNair stre...|
 Sports|Argentina #39;s Guillermo Canas won his third t...|
Sci/Tech| F5 bolsters firewall family The latest additio...|
Sci/Tech| Major Japanese computer maker Toshiba Corp. sa...|
Business|Microsoft Corp. (MSFT.O: Quote, Profile, Resear...|
 World| Israeli Prime Minister Ariel Sharon accused ar...|
Business| Shares of Sears, Roebuck and Co. were up abou...|
 Sports|Clemson, SC -- Reggie Merriweather rushed for 1...|
 World| Jonathan Adelstein, a former staffer for outgo...|
 World| Ukraine's outgoing president and opposition ap...|
 Sports|The prosecutor in Oakland County, Mich., today ...|
+--------+--------------------------------------------------+

In [0]:
trainDataset.count()


Out[29]: 14

In [0]:
from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
Sci/Tech| 4|
 Sports| 4|
Business| 3|
 World| 3|
+--------+-----+

In [0]:
testDataset = spark.read \
    .option("header", True) \
    .csv("/news_category_test.csv")\
    .sample(0.001) # For demonstration purposes only sample a fraction of the dataset

testDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
 World|Walking may protect the elderly from developing...|
 Sports|World No. 1 Vijay Singh, who is seeking his nin...|
Business| Stelco Inc. (STEa.TO) has lost a contract to s...|
 Sports|Deion Branch lists the Revolution among his fav...|
 World|Judges in the corruption trial of Silvio Berlus...|
+--------+--------------------------------------------------+

In [0]:
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 Sports| 2|
 World| 2|
Business| 1|
+--------+-----+

In [0]:
# if we want to split the dataset
'''(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)
print("Train Dataset Count: " + str(trainData.count()))
print("Test Dataset Count: " + str(testData.count()))'''


Out[33]: '(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)\nprint("Train Dataset Count: " + str(trainData.count()))\nprint("Test Dataset Count: " + str(testData.count()))'

In [0]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("lemma")


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]

### with Glove 100d embeddings

In [0]:
%fs mkdirs dbfs:/clf_glove_logs

res0: Boolean = true

In [0]:

glove_embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(2)\
      .setBatchSize(64) \
      .setLr(5e-3) \
      .setDropout(0.5)\
      .setEnableOutputLogs(True)\
      .setOutputLogsPath('dbfs:/clf_glove_logs')

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
'''
default classifierDL params:

maxEpochs -> 10,
lr -> 5e-3f,
dropout -> 0.5f,
batchSize -> 64,
enableOutputLogs -> false,
verbose -> Verbose.Silent.id,
validationSplit -> 0.0f,
outputLogsPath -> ""
'''

Out[36]: '\ndefault classifierDL params:\n\nmaxEpochs -> 10,\nlr -> 5e-3f,\ndropout -> 0.5f,\nbatchSize -> 64,\nenableOutputLogs -> false,\nverbose -> Verbose.Silent.id,\nvalidationSplit -> 0.0f,\noutputLogsPath -> ""\n'

In [0]:
# Train (2 min for 2 epochs)

clf_pipelineModel = clf_pipeline.fit(trainDataset)

In [0]:
%sh cd /dbfs/clf_glove_logs/ && ls -lt 

total 15
-rwxrwxrwx 1 root root 225 Mar 13 16:16 ClassifierDLApproach_4830a988a4c7.log
-rwxrwxrwx 1 root root 226 Mar 13 15:58 ClassifierDLApproach_4511444d9efe.log
-rwxrwxrwx 1 root root 226 Mar 13 15:28 ClassifierDLApproach_8f1f021f91e6.log
-rwxrwxrwx 1 root root 225 Mar 13 14:30 ClassifierDLApproach_5e70b1667f4a.log
-rwxrwxrwx 1 root root 727 Mar 13 11:11 ClassifierDLApproach_3445f63ac62b.log
-rwxrwxrwx 1 root root 226 Mar 13 10:59 ClassifierDLApproach_f0223fdca815.log
-rwxrwxrwx 1 root root 226 Mar 13 10:47 ClassifierDLApproach_6dca67da7b1d.log
-rwxrwxrwx 1 root root 226 Mar 10 18:32 ClassifierDLApproach_76e1aae85b04.log
-rwxrwxrwx 1 root root 223 Mar 10 13:30 ClassifierDLApproach_1b91f8975793.log
-rwxrwxrwx 1 root root 225 Mar 10 13:00 ClassifierDLApproach_55e8ffd0c132.log
-rwxrwxrwx 1 root root 226 Mar 10 12:29 ClassifierDLApproach_72ffd2fe88a2.log
-rwxrwxrwx 1 root root 223 Mar 10 12:17 ClassifierDLApproach_e2b93cfbc699.log
-rwxrwxrwx 1 root root 241 Mar 10 12:03 ClassifierDLApproach_0b1d09e34e58.log
-rwxrwxrwx 1 root root 230 Mar 7 13:38 ClassifierDLApproach_b98cc67dfaed.log
-rwxrwxrwx 1 root root 249 Mar 7 10:32 ClassifierDLApproach_28249ab6ba5f.log
-rwxrwxrwx 1 root root 248 Mar 6 20:13 ClassifierDLApproach_074eb4b37405.log
-rwxrwxrwx 1 root root 250 Mar 1 19:24 ClassifierDLApproach_38f25bc43bf0.log
-rwxrwxrwx 1 root root 249 Mar 1 19:23 ClassifierDLApproach_2159bef9e08c.log
-rwxrwxrwx 1 root root 247 Feb 22 17:36 ClassifierDLApproach_b654b69cbf34.log
-rwxrwxrwx 1 root root 250 Feb 13 19:27 ClassifierDLApproach_246e071395b1.log
-rwxrwxrwx 1 root root 251 Jan 18 21:21 ClassifierDLApproach_c9335482fadb.log
-rwxrwxrwx 1 root root 248 Jan 18 20:17 ClassifierDLApproach_2e90162f1943.log
-rwxrwxrwx 1 root root 246 Jan 18 19:39 ClassifierDLApproach_8f4e0a789b3b.log
-rwxrwxrwx 1 root root 247 Jan 18 17:46 ClassifierDLApproach_3116a9a8c042.log
-rwxrwxrwx 1 root root 251 Jan 18 15:52 ClassifierDLApproach_a5f7c28bed0a.log
-rwxrwxrwx 1 root root 250 Jan 18 15:37 ClassifierDLApproach_1cbacf770d39.log
-rwxrwxrwx 1 root root 247 Jan 8 18:49 ClassifierDLApproach_b22b3b8f3599.log
-rwxrwxrwx 1 root root 242 Oct 10 16:39 ClassifierDLApproach_0538f27c7737.log
-rwxrwxrwx 1 root root 247 Feb 8 2022 ClassifierDLApproach_2f4af8feed66.log

In [0]:
%sh cat /dbfs/clf_glove_logs/* 

Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 7.30s - loss: 1627.6475 - acc: 0.8691 - batches: 1875
Epoch 1/2 - 8.18s - loss: 1609.324 - acc: 0.88045 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 11.39s - loss: 1631.7356 - acc: 0.868675 - batches: 1875
Epoch 1/2 - 7.65s - loss: 1613.3915 - acc: 0.8811333 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 107 - classes: 4
Epoch 0/2 - 0.41s - loss: 2.4441724 - acc: 0.63880813 - batches: 2
Epoch 1/2 - 0.01s - loss: 2.7519903 - acc: 0.96511626 - batches: 2
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 9 - classes: 3
Epoch 0/2 - 0.40s - loss: 1.101165 - acc: 1.0 - batches: 1
Epoch 1/2 - 0.00s - loss: 0.949149 - acc: 1.0 - batches: 1
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 9.98s - loss: 1647.9044 - acc: 0.86836666 - batches: 1875
Epoch 1/2 - 9.52s - loss: 1620.5863 - acc: 0.88215834 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 10.20s - loss: 1629.3923 - acc: 0.8687417 - batches: 1875
Epoch 1/2 - 9.54s - loss: 1597.7142 - acc: 0.8807167 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 10.77s - loss: 1630.2042 - acc: 0.86755836 - batches: 1875
Epoch 1/2 - 10.06s - loss: 1597.9321 - acc: 0.880625 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 7.81s - loss: 1632.1327 - acc: 0.8692833 - batches: 1875
Epoch 1/2 - 7.29s - loss: 1609.1466 - acc: 0.88150835 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 11.02s - loss: 1651.4069 - acc: 0.8674333 - batches: 1875
Epoch 1/2 - 10.15s - loss: 1627.0297 - acc: 0.88105 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 20.63s - loss: 1640.8435 - acc: 0.8707 - batches: 1875
Epoch 1/2 - 19.17s - loss: 1609.2996 - acc: 0.8815917 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 8.19s - loss: 1632.2864 - acc: 0.86825 - batches: 1875
Epoch 1/2 - 7.46s - loss: 1609.3625 - acc: 0.88009167 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 13 - classes: 4
Epoch 0/2 - 0.42s - loss: 2.7408495 - acc: 0.8 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, recall: 0.0, f1: 0.0
Epoch 1/2 - 0.01s - loss: 2.4157548 - acc: 1.0 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.022727273, rec: 0.125, f1: 0.038461536
Micro-average prec: 0.083333336, recall: 0.083333336, f1: 0.083333336
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 16 - classes: 4
Epoch 0/2 - 0.38s - loss: 1.3749708 - acc: 1.0 - batches: 1
Epoch 1/2 - 0.00s - loss: 1.3438281 - acc: 1.0 - batches: 1
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 11.16s - loss: 1670.2426 - acc: 0.8623083 - batches: 1875
Epoch 1/2 - 10.14s - loss: 1633.9469 - acc: 0.8805917 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 15 - classes: 3
Epoch 0/2 - 0.43s - loss: 1.1156888 - acc: 1.0 - batches: 1
Epoch 1/2 - 0.01s - loss: 1.1135736 - acc: 1.0 - batches: 1
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 14 - classes: 4
E

In [0]:
# get the predictions on test Set

preds = clf_pipelineModel.transform(testDataset)

preds.select('category','description',"class.result").show(10, truncate=80)

+--------+--------------------------------------------------------------------------------+----------+
category| description| result|
+--------+--------------------------------------------------------------------------------+----------+
 World| Walking may protect the elderly from developing dementia, research suggests.|[Sci/Tech]|
 Sports|World No. 1 Vijay Singh, who is seeking his ninth win on the PGA Tour this ye...| [Sports]|
Business| Stelco Inc. (STEa.TO) has lost a contract to supply steel to its biggest cus...|[Sci/Tech]|
 Sports|Deion Branch lists the Revolution among his favorite soccer teams in his pres...| [Sports]|
 World|Judges in the corruption trial of Silvio Berlusconi withdrew yesterday to dec...| [World]|
+--------+--------------------------------------------------------------------------------+----------+

In [0]:
# due to bug in cluster mode (https://github.com/JohnSnowLabs/spark-nlp/issues/857) , as a workaround, you can just save the fitted model and then load back from dbfs and then transform on the test set. 
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')
classsifierdlmodel_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')


clf_pipeline_pred = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            clf_pipelineModel.stages[-2],
            classsifierdlmodel_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = clf_pipeline_pred.fit(empty_data).transform(testDataset)

In [0]:
preds_df = result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))


/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 Business 0.00 0.00 0.00 0
 Sci/Tech 0.00 0.00 0.00 2
 Sports 1.00 1.00 1.00 2
 World 0.50 1.00 0.67 1

 accuracy 0.60 5
 macro avg 0.38 0.50 0.42 5
weighted avg 0.50 0.60 0.53 5

### with Bert Embeddings

In [0]:
document_assembler = DocumentAssembler() \
                .setInputCol("description") \
                .setOutputCol("document")

tokenizer = Tokenizer() \
                .setInputCols(["document"]) \
                .setOutputCol("token")
      
bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
                .setInputCols(["document",'token'])\
                .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
                .setInputCols(["document", "embeddings"]) \
                .setOutputCol("sentence_embeddings") \
                .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("category")\
                .setMaxEpochs(2)\
                .setLr(0.001)\
                .setBatchSize(8)\
                .setEnableOutputLogs(True)\
                .setOutputLogsPath('dbfs:/clf_bert_logs')

bert_clf_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence,
                                     classsifierdl])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[ | ][ / ][ — ][OK!]

In [0]:
# remove the existing logs

! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory

In [0]:
# training will take some time due to Bert (use GPU runtime when possible)

bert_clf_pipelineModel = bert_clf_pipeline.fit(trainDataset)

In [0]:
%sh cd /dbfs/clf_bert_logs/ && ls -lt 

total 16
-rwxrwxrwx 1 root root 229 Mar 13 16:17 ClassifierDLApproach_e2c73ea5ba5a.log
-rwxrwxrwx 1 root root 230 Mar 13 15:29 ClassifierDLApproach_d4bd54b1cf4a.log
-rwxrwxrwx 1 root root 224 Mar 13 14:31 ClassifierDLApproach_7d88dda4897d.log
-rwxrwxrwx 1 root root 228 Mar 13 11:12 ClassifierDLApproach_a0b313f8e145.log
-rwxrwxrwx 1 root root 226 Mar 13 10:59 ClassifierDLApproach_fcef04fd805a.log
-rwxrwxrwx 1 root root 225 Mar 13 10:48 ClassifierDLApproach_b96839660621.log
-rwxrwxrwx 1 root root 227 Mar 10 18:33 ClassifierDLApproach_c8f72b7af29f.log
-rwxrwxrwx 1 root root 226 Mar 10 13:31 ClassifierDLApproach_f9993c7052af.log
-rwxrwxrwx 1 root root 713 Mar 10 13:01 ClassifierDLApproach_c56b14c58a3d.log
-rwxrwxrwx 1 root root 712 Mar 10 12:29 ClassifierDLApproach_0ebc646077cd.log
-rwxrwxrwx 1 root root 785 Mar 10 12:04 ClassifierDLApproach_66252fbd444a.log
-rwxrwxrwx 1 root root 772 Mar 7 13:40 ClassifierDLApproach_f2b573a46015.log
-rwxrwxrwx 1 root root 821 Mar 7 10:45 ClassifierDLApproach_cd29f7496a9e.log
-rwxrwxrwx 1 root root 825 Feb 22 17:54 ClassifierDLApproach_714cbd405cf5.log
-rwxrwxrwx 1 root root 827 Feb 13 19:44 ClassifierDLApproach_7a1b9eb6cfc4.log
-rwxrwxrwx 1 root root 824 Jan 18 21:39 ClassifierDLApproach_393c04fdc101.log
-rwxrwxrwx 1 root root 833 Jan 18 20:35 ClassifierDLApproach_277cc0b5bbb1.log
-rwxrwxrwx 1 root root 823 Jan 18 19:57 ClassifierDLApproach_2ff3eb4259e4.log
-rwxrwxrwx 1 root root 823 Jan 18 17:59 ClassifierDLApproach_3c5b7e1632d7.log
-rwxrwxrwx 1 root root 253 Feb 4 2022 ClassifierDLApproach_3bf510f9e159.log

In [0]:
%sh cat /dbfs/clf_bert_logs/* 

Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 13 - classes: 4
Epoch 0/10 - 0.33s - loss: 2.904717 - acc: 0.2 - batches: 2
Epoch 1/10 - 0.01s - loss: 2.724072 - acc: 1.0 - batches: 2
Epoch 2/10 - 0.01s - loss: 2.66022 - acc: 1.0 - batches: 2
Epoch 3/10 - 0.01s - loss: 2.6134655 - acc: 1.0 - batches: 2
Epoch 4/10 - 0.01s - loss: 2.5776033 - acc: 1.0 - batches: 2
Epoch 5/10 - 0.01s - loss: 2.5494819 - acc: 1.0 - batches: 2
Epoch 6/10 - 0.01s - loss: 2.5269456 - acc: 1.0 - batches: 2
Epoch 7/10 - 0.01s - loss: 2.5085502 - acc: 1.0 - batches: 2
Epoch 8/10 - 0.01s - loss: 2.4931414 - acc: 1.0 - batches: 2
Epoch 9/10 - 0.01s - loss: 2.4805684 - acc: 1.0 - batches: 2
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 51.26s - loss: 13316.869 - acc: 0.85573334 - batches: 15000
Epoch 1/10 - 53.15s - loss: 13111.651 - acc: 0.869975 - batches: 15000
Epoch 2/10 - 50.40s - loss: 13015.457 - acc: 0.8773583 - batches: 15000
Epoch 3/10 - 49.07s - loss: 12962.077 - acc: 0.88266665 - batches: 15000
Epoch 4/10 - 51.60s - loss: 12914.85 - acc: 0.8865917 - batches: 15000
Epoch 5/10 - 51.42s - loss: 12872.899 - acc: 0.8895083 - batches: 15000
Epoch 6/10 - 51.73s - loss: 12834.163 - acc: 0.89238334 - batches: 15000
Epoch 7/10 - 50.52s - loss: 12796.696 - acc: 0.89475834 - batches: 15000
Epoch 8/10 - 50.58s - loss: 12765.013 - acc: 0.89685833 - batches: 15000
Epoch 9/10 - 50.21s - loss: 12738.552 - acc: 0.8986167 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 49.91s - loss: 13335.299 - acc: 0.855975 - batches: 15000
Epoch 1/10 - 48.68s - loss: 13120.338 - acc: 0.8703833 - batches: 15000
Epoch 2/10 - 48.95s - loss: 13020.424 - acc: 0.8771167 - batches: 15000
Epoch 3/10 - 48.48s - loss: 12957.696 - acc: 0.88184166 - batches: 15000
Epoch 4/10 - 50.16s - loss: 12916.484 - acc: 0.885625 - batches: 15000
Epoch 5/10 - 48.50s - loss: 12878.053 - acc: 0.8884417 - batches: 15000
Epoch 6/10 - 48.61s - loss: 12841.42 - acc: 0.8913417 - batches: 15000
Epoch 7/10 - 49.39s - loss: 12810.181 - acc: 0.8936 - batches: 15000
Epoch 8/10 - 48.43s - loss: 12783.97 - acc: 0.8955333 - batches: 15000
Epoch 9/10 - 48.68s - loss: 12758.514 - acc: 0.897325 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 51.98s - loss: 13350.956 - acc: 0.8552333 - batches: 15000
Epoch 1/10 - 51.24s - loss: 13136.743 - acc: 0.86975 - batches: 15000
Epoch 2/10 - 51.09s - loss: 13025.483 - acc: 0.8770083 - batches: 15000
Epoch 3/10 - 50.69s - loss: 12953.085 - acc: 0.881825 - batches: 15000
Epoch 4/10 - 51.12s - loss: 12895.694 - acc: 0.8860083 - batches: 15000
Epoch 5/10 - 50.27s - loss: 12847.632 - acc: 0.8893667 - batches: 15000
Epoch 6/10 - 53.71s - loss: 12808.43 - acc: 0.89205 - batches: 15000
Epoch 7/10 - 54.83s - loss: 12777.251 - acc: 0.8941583 - batches: 15000
Epoch 8/10 - 53.62s - loss: 12748.877 - acc: 0.89641666 - batches: 15000
Epoch 9/10 - 53.26s - loss: 12724.57 - acc: 0.8979333 - batches: 15000
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/2 - 42.96s - loss: 19865.465 - acc: 0.32381666 - batches: 15000
Epoch 1/2 - 46.85s - loss: 19814.223 - acc: 0.32563335 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 38.61s - loss: 13311.776 - acc: 0.8552333 - batches: 15000
Epoch 1/10 - 38.72s - loss: 13094.07 - acc: 0.8702 - batches: 15000
Epoch 2/10 - 38.01s - loss: 13000.639 - acc: 0.87731665 - batches: 15000
Epoch 3/10 - 39.03s - loss: 12937.432 - acc: 0.88226664 - batches: 15000
Epoch 4/10 - 38.70s - loss: 12886.709 - acc: 0.8860083 - batches: 15000
Epoch 5/10 - 37.93s - loss: 12841.292 - acc: 0.88915 - batches: 15000
Epoch 6/10 - 37.2

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

preds = bert_clf_pipelineModel.transform(testDataset)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 Business 0.00 0.00 0.00 1
 Sci/Tech 0.00 0.00 0.00 0
 Sports 0.67 1.00 0.80 2
 World 0.00 0.00 0.00 2

 accuracy 0.40 5
 macro avg 0.17 0.25 0.20 5
weighted avg 0.27 0.40 0.32 5

#### testDataset Parameter

In [0]:
embedding_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence])

In [0]:
test_data = embedding_pipeline.fit(testDataset).transform(testDataset)

In [0]:
test_data.printSchema()

root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
test_data.write.mode("overwrite").parquet('dbfs:/test_data.parquet')

In [0]:
classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("category")\
                .setMaxEpochs(2)\
                .setLr(0.001)\
                .setBatchSize(8)\
                .setEnableOutputLogs(True)\
                .setTestDataset("dbfs:/test_data.parquet")\
                .setOutputLogsPath('clf_logs')

clf_pipeline = Pipeline(stages=[document_assembler,
                                tokenizer,
                                bert_embeddings,
                                embeddingsSentence,
                                classsifierdl])

In [0]:
%%time
clf_model = clf_pipeline.fit(trainDataset)

CPU times: user 94.7 ms, sys: 28.3 ms, total: 123 ms
Wall time: 14.3 s

In [0]:
from sklearn.metrics import classification_report

preds = clf_model.transform(testDataset)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 Business 0.00 0.00 0.00 1
 Sci/Tech 0.00 0.00 0.00 0
 Sports 0.67 1.00 0.80 2
 World 0.00 0.00 0.00 2

 accuracy 0.40 5
 macro avg 0.17 0.25 0.20 5
weighted avg 0.27 0.40 0.32 5

### Getting prediction from Trained model

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(clf_pipelineModel)

In [0]:
text='''
Fearing the fate of Italy, the centre-right government has threatened to be merciless with those who flout tough restrictions. 
As of Wednesday it will also include all shops being closed across Greece, with the exception of supermarkets. Banks, pharmacies, pet-stores, mobile phone stores, opticians, bakers, mini-markets, couriers and food delivery outlets are among the few that will also be allowed to remain open.
'''
result = light_model.annotate(text)

result['class']

Out[57]: ['Sci/Tech']

In [0]:
light_model.annotate('the soccer games will be postponed.')

Out[58]: {'lemma': ['soccer', 'game', 'postpone'],
 'document': ['the soccer games will be postponed.'],
 'normalized': ['the', 'soccer', 'games', 'will', 'be', 'postponed'],
 'sentence_embeddings': ['the soccer games will be postponed.'],
 'cleanTokens': ['soccer', 'games', 'postponed'],
 'token': ['the', 'soccer', 'games', 'will', 'be', 'postponed', '.'],
 'class': ['Sports'],
 'embeddings': ['soccer', 'game', 'postpone']}

## ClassifierDL with Universal Sentence Embeddings

In [0]:
%fs mkdirs dbfs:/clf_use_logs

res1: Boolean = true

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
      .setInputCol("description")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
     .setInputCols(["document"])\
     .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(2)\
      .setLr(0.001)\
      .setBatchSize(8)\
      .setEnableOutputLogs(True)\
      .setOutputLogsPath('dbfs:/clf_use_logs')

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
# 5 epochs takes around 8 min

use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [0]:
%sh cd  /dbfs/clf_use_logs/ && ls -l


total 16
-rwxrwxrwx 1 root root 691 Mar 13 16:18 ClassifierDLApproach_00122a5f6b4a.log
-rwxrwxrwx 1 root root 463 Jan 18 20:41 ClassifierDLApproach_1c04c9f52857.log
-rwxrwxrwx 1 root root 758 Mar 13 11:13 ClassifierDLApproach_2380a05a8223.log
-rwxrwxrwx 1 root root 2826 Mar 10 12:05 ClassifierDLApproach_29c0b9b9b5c3.log
-rwxrwxrwx 1 root root 2512 Mar 10 13:01 ClassifierDLApproach_561ee2d6380b.log
-rwxrwxrwx 1 root root 461 Jan 18 18:04 ClassifierDLApproach_7fd7a2d21889.log
-rwxrwxrwx 1 root root 467 Jan 18 15:43 ClassifierDLApproach_80ccd326ebeb.log
-rwxrwxrwx 1 root root 464 Oct 10 16:50 ClassifierDLApproach_99f47a5282fc.log
-rwxrwxrwx 1 root root 462 Jan 8 18:54 ClassifierDLApproach_ae9c3cff1971.log
-rwxrwxrwx 1 root root 757 Mar 10 13:32 ClassifierDLApproach_c1cb0cf34d93.log
-rwxrwxrwx 1 root root 773 Mar 13 15:30 ClassifierDLApproach_d3165f014683.log
-rwxrwxrwx 1 root root 462 Feb 8 2022 ClassifierDLApproach_d375b9652a4d.log
-rwxrwxrwx 1 root root 2580 Mar 10 12:30 ClassifierDLApproach_f46716a295a8.log
-rwxrwxrwx 1 root root 460 Jan 18 20:03 ClassifierDLApproach_ff79577dff7b.log

In [0]:
%sh cat /dbfs/clf_use_logs/*

Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 14 - classes: 4
Epoch 0/2 - 0.43s - loss: 2.8353834 - acc: 0.33333334 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.5, rec: 0.5, f1: 0.5
Micro-average prec: 0.6, recall: 0.6, f1: 0.6
Epoch 1/2 - 0.03s - loss: 2.7504845 - acc: 1.0 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.5, rec: 0.5, f1: 0.5
Micro-average prec: 0.6, recall: 0.6, f1: 0.6
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 14 - classes: 4
Epoch 0/2 - 0.52s - loss: 2.7644954 - acc: 1.0 - batches: 2
Epoch 1/2 - 0.01s - loss: 2.7081473 - acc: 1.0 - batches: 2
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 50.39s - loss: 12932.341 - acc: 0.8826917 - batches: 15000
Epoch 1/5 - 51.33s - loss: 12771.584 - acc: 0.8914667 - batches: 15000
Epoch 2/5 - 51.19s - loss: 12704.802 - acc: 0.89598334 - batches: 15000
Epoch 3/5 - 49.36s - loss: 12654.659 - acc: 0.8996083 - batches: 15000
Epoch 4/5 - 49.81s - loss: 12618.718 - acc: 0.90205 - batches: 15000
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 16 - classes: 4
Epoch 0/2 - 0.43s - loss: 2.6717908 - acc: 0.5 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.041666668, rec: 0.25, f1: 0.071428575
Micro-average prec: 0.14285715, recall: 0.14285715, f1: 0.14285715
Epoch 1/2 - 0.01s - loss: 2.5469956 - acc: 0.875 - batches: 2
Quality on test dataset: 
Macro-average prec: 0.225, rec: 0.5, f1: 0.31034482
Micro-average prec: 0.42857143, recall: 0.42857143, f1: 0.42857143
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 16 - classes: 4
Epoch 0/2 - 0.41s - loss: 2.766152 - acc: 0.1875 - batches: 2
Epoch 1/2 - 0.01s - loss: 2.7169428 - acc: 0.875 - batches: 2
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 107 - classes: 4
Epoch 0/10 - 0.40s - loss: 17.836956 - acc: 0.49358973 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.11111111, rec: 0.33333334, f1: 0.16666667
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 1/10 - 0.05s - loss: 15.356168 - acc: 0.86858976 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 2/10 - 0.04s - loss: 13.36192 - acc: 0.94551283 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 3/10 - 0.04s - loss: 12.640213 - acc: 0.94551283 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 4/10 - 0.04s - loss: 12.220889 - acc: 0.95512825 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 5/10 - 0.05s - loss: 12.026598 - acc: 0.94551283 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 6/10 - 0.04s - loss: 11.8906975 - acc: 0.9647436 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 7/10 - 0.04s - loss: 11.799029 - acc: 0.9647436 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.16666667, recall: 0.16666667, f1: 0.16666667
Epoch 8/10 - 0.04s - loss: 11.732265 - acc: 0.9839744 - batches: 14
Quality on test dataset: 
Macro-average prec: 0.16666667, rec: 0.33333334, f1: 0.22222224
Micro-average prec: 0.166

### Saving & loading back the trained model

In [0]:
use_pipelineModel.stages

Out[63]: [DocumentAssembler_ad08b3b1a65a,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_1d296d49c35d]

In [0]:
use_pipelineModel.stages[2].write().overwrite().save('/databricks/driver/models/ClassifierDL_USE_e5')

In [0]:
classsifierdlmodel = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_USE_e5')

In [0]:
clf_lp = get_clf_lp('dbfs:/databricks/driver/models/ClassifierDL_USE_e5', sentiment_dl=False,  pretrained=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classes: ['Sci/Tech', 'Business', 'Sports', 'World']

In [0]:
clf_lp.annotate(text)['class']

Out[67]: ['Sci/Tech']

# SentimentDL Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb`

In [0]:
!wget -q aclimdb_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_train.csv
!wget -q aclimdb_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_test.csv
  
dbutils.fs.cp("file:/databricks/driver/aclimdb_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/aclimdb_test.csv", "dbfs:/")

Out[68]: True

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/aclimdb_train.csv") \
      .sample(0.02) # For demonstration purposes only sample a fraction of the dataset

trainDataset.show(10)
trainDataset.count()

+--------------------+--------+
 text| label|
+--------------------+--------+
Difficult film to...|positive|
In short, this mo...|negative|
I cannot see why ...|negative|
I really can't sa...|positive|
I really enjoyed ...|positive|
If you haven't fi...|negative|
Probably the fine...|positive|
Mix exotic tropic...|negative|
In what I can say...|negative|
I saw this film y...|negative|
+--------------------+--------+
only showing top 10 rows

Out[69]: 526

In [0]:
testDataset = spark.read \
      .option("header", True) \
      .csv("/aclimdb_test.csv") \
      .sample(0.02)

testDataset.show(10)
testDataset.count()

+--------------------+--------+
 text| label|
+--------------------+--------+
It's hard to beli...|negative|
I saw this film f...|negative|
There have been s...|positive|
Visconti's master...|positive|
Let me just say -...|positive|
As other reviewer...|negative|
This movie had po...|negative|
An interesting co...|positive|
OK, if you would ...|positive|
Charlie Wilson's ...|positive|
+--------------------+--------+
only showing top 10 rows

Out[70]: 546

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
   .setInputCols(["document"])\
   .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(2)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(stages = [
    document,
    use,
    sentimentdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

In [0]:
result = pipelineModel.transform(testDataset)

result_df = result.select('text','label',"class.result").toPandas()

result_df.head(10)

Out[73]:

,text,label,result
0,"It's hard to believe that a movie this bad wasn't produced once, but four times! Most movies req...",negative,[negative]
1,"I saw this film for one reason: the tagline is ""Upset the head and you're dead!"" Cracking. It's...",negative,[negative]
2,"There have been several comments already on the site focusing on the ""prestige"" feel of the film...",positive,[positive]
3,Visconti's masterpiece! I admit that I am unfamiliar with much of his work but I cannot imagine ...,positive,[positive]
4,Let me just say - I love the horror genre to the extent that I see every single one that I can g...,positive,[positive]
5,"As other reviewers have noted, this movie is a cross between (i.e. stolen from) stories we have ...",negative,[negative]
6,This movie had potential. If it had been handled differently. What it needed was a different dir...,negative,[negative]
7,"An interesting comedy, taking place on a train from Stockholm to Berlin, December 1945. One can'...",positive,[positive]
8,"OK, if you would judge the movie to now a days it wouldn't fit in to well.If you watched FI now ...",positive,[positive]
9,Charlie Wilson's war is an excellent example of how films should be made. This movie is of the h...,positive,[positive]


# Multilabel Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/MultiClassifierDL_train_multi_label_toxic_classifier.ipynb`

In [0]:
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_train.snappy.parquet'
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_test.snappy.parquet'

dbutils.fs.cp("file:/databricks/driver/toxic_train.snappy.parquet", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/toxic_test.snappy.parquet", "dbfs:/")

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 2702k 100 2702k 0 0 16.1M 0 --:--:-- --:--:-- --:--:-- 16.1M
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 289k 100 289k 0 0 2520k 0 --:--:-- --:--:-- --:--:-- 2520k
Out[74]: True

In [0]:
trainDataset = spark.read.parquet("dbfs:/toxic_train.snappy.parquet").sample(0.02)
testDataset = spark.read.parquet("dbfs:/toxic_train.snappy.parquet").sample(0.02)

print(trainDataset.count())
print(testDataset.count())

298
274

In [0]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")\
  .setCleanupMode("shrink")

# Here we use the state-of-the-art Universal Sentence Encoder model from TF Hub
embeddings = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"])\
  .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("category")\
  .setLabelColumn("labels")\
  .setBatchSize(128)\
  .setMaxEpochs(2)\
  .setLr(1e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        multiClassifier
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

In [0]:
preds = pipelineModel.transform(testDataset)
preds_df = preds.select('text','labels',"category.result").toPandas()
preds_df.head(10)

Out[78]:

,text,labels,result
0,God is dead\nI don't mean to startle anyone but God is dead. We should not worry about him anymo...,[toxic],"[toxic, insult, obscene]"
1,"i have a dick, its bigger than yours! hahaha","[toxic, severe_toxic, obscene]","[toxic, insult, obscene]"
2,The only vandals are pathetic wiki administrators as encyclopediadramatica.com says right!,[toxic],"[toxic, insult]"
3,"""\n\n SHUT THE FUCK UP \n\n SHUT THE FUCK UP You bother me again, there will be problems ""","[toxic, obscene]","[toxic, insult, obscene]"
4,"You can't kill the party Soap, YOU CAN'T KILL THE PARTY!",[toxic],[toxic]
5,DreadedWalrus thinks you are gay. \n\nDreadedWalrus thinks you are gay.,[toxic],"[toxic, insult, obscene]"
6,picking on a 10 year old!!! soooo sad \n\nJust shut up okay?Im only 10 years old. I just wanted ...,"[toxic, insult]","[toxic, insult, obscene]"
7,"Hi. Welcome me to nothing bitch, I've been using Wikipedia for over 3 years. I don't vandalise...","[toxic, obscene, insult]","[toxic, insult]"
8,how is pointing out that you are ignorant a personal attack? I didn't make this up. It isn't y...,"[toxic, insult]","[toxic, insult, obscene]"
9,I'm sure you eat alot of tube steak,[toxic],"[toxic, insult, obscene]"


## Multilabel Classifier with Bert Embeddings

In [0]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
              .setInputCol("text")\
              .setOutputCol("document")\
              .setCleanupMode("shrink")

bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
              .setInputCols(["document"])\
              .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset

multiClassifier = MultiClassifierDLApproach()\
              .setInputCols("sentence_embeddings")\
              .setOutputCol("category")\
              .setLabelColumn("labels")\
              .setBatchSize(128)\
              .setMaxEpochs(2)\
              .setLr(1e-3)\
              .setThreshold(0.5)\
              .setShufflePerEpoch(False)\
              .setEnableOutputLogs(True)\
              .setValidationSplit(0.1)

pipeline = Pipeline(stages = [document,
                              bert_sent,
                              multiClassifier])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
%%time
pipelineModel = pipeline.fit(trainDataset)

CPU times: user 321 ms, sys: 121 ms, total: 442 ms
Wall time: 1min 26s

In [0]:
preds = pipelineModel.transform(testDataset)

In [0]:
preds_df = preds.select('text','labels',"category.result").toPandas()

In [0]:
preds_df.head(10)

Out[83]:

,text,labels,result
0,God is dead\nI don't mean to startle anyone but God is dead. We should not worry about him anymo...,[toxic],[toxic]
1,"i have a dick, its bigger than yours! hahaha","[toxic, severe_toxic, obscene]","[toxic, insult, obscene]"
2,The only vandals are pathetic wiki administrators as encyclopediadramatica.com says right!,[toxic],[toxic]
3,"""\n\n SHUT THE FUCK UP \n\n SHUT THE FUCK UP You bother me again, there will be problems ""","[toxic, obscene]","[toxic, insult, obscene]"
4,"You can't kill the party Soap, YOU CAN'T KILL THE PARTY!",[toxic],"[toxic, insult, obscene]"
5,DreadedWalrus thinks you are gay. \n\nDreadedWalrus thinks you are gay.,[toxic],[toxic]
6,picking on a 10 year old!!! soooo sad \n\nJust shut up okay?Im only 10 years old. I just wanted ...,"[toxic, insult]","[toxic, insult]"
7,"Hi. Welcome me to nothing bitch, I've been using Wikipedia for over 3 years. I don't vandalise...","[toxic, obscene, insult]",[toxic]
8,how is pointing out that you are ignorant a personal attack? I didn't make this up. It isn't y...,"[toxic, insult]",[toxic]
9,I'm sure you eat alot of tube steak,[toxic],"[toxic, insult, obscene]"


# Case Study: Conference Title Classification

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/title_conference.csv
  
dbutils.fs.cp("file:/databricks/driver/title_conference.csv", "dbfs:/")

Out[84]: True

In [0]:
import pandas as pd
df = pd.read_csv('title_conference.csv')
df

Out[85]:

,Title,Conference
0,Innovation in Database Management: Computer Science vs. Engineering.,VLDB
1,High performance prime field multiplication for GPU.,ISCAS
2,enchanted scissors: a scissor interface for support in cutting and interactive fabrication.,SIGGRAPH
3,Detection of channel degradation attack by Intermediary Node in Linear Networks.,INFOCOM
4,Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS
...,...,...
2502,A new QR-decomposition based recursive frequency estimator for multiple sinusoids in impulsive n...,ISCAS
2503,CNN Implementation of Spin Filters for Electronic Speckle Pattern Interferometry Applications.,ISCAS
2504,FaceKit: A Database Interface Design Toolkit.,VLDB
2505,On the trade-off between the number of scrolls and the operating frequency of the chaotic attrac...,ISCAS


In [0]:
df.Conference.value_counts()

Out[86]: ISCAS 864
INFOCOM 515
VLDB 423
WWW 379
SIGGRAPH 326
Name: Conference, dtype: int64

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/title_conference.csv") \
      .sample(0.1)

(trainingData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))


Training Dataset Count: 198
Test Dataset Count: 52

In [0]:
document = DocumentAssembler()\
      .setInputCol("Title")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Conference")\
      .setMaxEpochs(2)\
      .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
use_pipelineModel = use_clf_pipeline.fit(trainingData)

In [0]:
use_pipelineModel.stages

Out[90]: [DocumentAssembler_7b59be4bb921,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_3f024177ad11]

In [0]:
use_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/use_clf')
use_clf_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/use_clf')

use_clf_pipeline_pred = Pipeline(
    stages=[document, 
            use_pipelineModel.stages[-2],
            use_clf_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = use_clf_pipeline_pred.fit(empty_data).transform(testData)

In [0]:
result.select('Title','Conference',"class.result").show(10, truncate=80)


+--------------------------------------------------------------------------------+----------+-------+
 Title|Conference| result|
+--------------------------------------------------------------------------------+----------+-------+
 A 10-bit 2GHz Current-Steering CMOS D/A Converter.| ISCAS|[ISCAS]|
 A Pragmatic Approach to Structured Database Design.| VLDB|[ISCAS]|
 A click model for time-sensitive queries.| WWW|[ISCAS]|
 A framework for evaluating network measures for functional importance.| WWW|[ISCAS]|
A low-power, high-speed RB-to-NB converter for fast redundant binary multiplier.| ISCAS|[ISCAS]|
 A new model for adaptive displays based on von Kries hypothesis.| SIGGRAPH|[ISCAS]|
A-ToPSS: A Publish/Subscribe System Supporting Imperfect Information Processing.| VLDB|[ISCAS]|
Accurate transient response model for automatic synthesis of high-speed opera...| ISCAS|[ISCAS]|
 Advances in semantic multimedia analysis for personalised content access.| ISCAS|[ISCAS]|
 Almost optimal virtual machine placement for traffic intense data centers.| INFOCOM|[ISCAS]|
+--------------------------------------------------------------------------------+----------+-------+
only showing top 10 rows

In [0]:
# We are going to use sklearn to evalute the results on test dataset
preds_df = result.select('Conference','Title',"class.result").toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['Conference']))


/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 INFOCOM 0.00 0.00 0.00 0
 ISCAS 1.00 0.35 0.51 52
 SIGGRAPH 0.00 0.00 0.00 0
 VLDB 0.00 0.00 0.00 0
 WWW 0.00 0.00 0.00 0

 accuracy 0.35 52
 macro avg 0.20 0.07 0.10 52
weighted avg 1.00 0.35 0.51 52

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

preds = pd.DataFrame(confusion_matrix(preds_df['result'], preds_df['Conference']), columns = np.unique(preds_df['Conference']), index =  np.unique(preds_df['Conference']))
preds

Out[94]:

,INFOCOM,ISCAS,SIGGRAPH,VLDB,WWW
INFOCOM,0,0,0,0,0
ISCAS,7,18,9,12,6
SIGGRAPH,0,0,0,0,0
VLDB,0,0,0,0,0
WWW,0,0,0,0,0


### Bert Sentence Embeddings

In [0]:
document = DocumentAssembler()\
              .setInputCol("Title")\
              .setOutputCol("document")
    
bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
              .setInputCols(["document"])\
              .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
              .setInputCols(["sentence_embeddings"])\
              .setOutputCol("class")\
              .setLabelColumn("Conference")\
              .setMaxEpochs(2)\
              .setLr(0.001)\
              .setBatchSize(8)\
              .setEnableOutputLogs(True)

bert_clf_pipeline = Pipeline(stages = [document,
                                      bert_sent,
                                      classsifierdl])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[ | ][OK!]

In [0]:
%%time
bert_pipelineModel = bert_clf_pipeline.fit(trainingData)

CPU times: user 62.2 ms, sys: 36.3 ms, total: 98.5 ms
Wall time: 14.8 s

In [0]:
from sklearn.metrics import classification_report

preds = bert_pipelineModel.transform(testData)

preds_df = preds.select('Conference',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['Conference'], preds_df['result']))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 INFOCOM 0.00 0.00 0.00 7
 ISCAS 0.45 0.94 0.61 18
 SIGGRAPH 0.36 0.56 0.43 9
 VLDB 0.00 0.00 0.00 12
 WWW 0.00 0.00 0.00 6

 accuracy 0.42 52
 macro avg 0.16 0.30 0.21 52
weighted avg 0.22 0.42 0.29 52

# Save model and Zip it for Modelshub Upload/Downloads

In [0]:
# Save a Spark NLP pipeline
bert_clf_pipeline.write().overwrite().save('file:/databricks/driver/models/my_nlp_pipeline')

In [0]:
# cd into saved dir and zip
! cd /databricks/driver/models/my_nlp_pipeline && zip -r my_nlp_pipeline.zip *

adding: metadata/ (stored 0%)
 adding: metadata/._SUCCESS.crc (stored 0%)
 adding: metadata/_SUCCESS (stored 0%)
 adding: stages/ (stored 0%)
 adding: stages/0_DocumentAssembler_d0dc8a5a357c/ (stored 0%)
 adding: stages/0_DocumentAssembler_d0dc8a5a357c/metadata/ (stored 0%)
 adding: stages/0_DocumentAssembler_d0dc8a5a357c/metadata/._SUCCESS.crc (stored 0%)
 adding: stages/0_DocumentAssembler_d0dc8a5a357c/metadata/_SUCCESS (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/ (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/bert_sentence_tensorflow (deflated 8%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/metadata/ (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/metadata/._SUCCESS.crc (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/metadata/_SUCCESS (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/.bert_sentence_tensorflow.crc (deflated 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/fields/ (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/fields/vocabulary/ (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/fields/vocabulary/._SUCCESS.crc (stored 0%)
 adding: stages/1_BERT_SENTENCE_EMBEDDINGS_3608c0d843af/fields/vocabulary/_SUCCESS (stored 0%)
 adding: stages/2_ClassifierDLApproach_d524bf851d88/ (stored 0%)
 adding: stages/2_ClassifierDLApproach_d524bf851d88/metadata/ (stored 0%)
 adding: stages/2_ClassifierDLApproach_d524bf851d88/metadata/._SUCCESS.crc (stored 0%)
 adding: stages/2_ClassifierDLApproach_d524bf851d88/metadata/_SUCCESS (stored 0%)

End of Notebook # 4